## Implementing granular access control to Offline Feature Store and Feature Groups using AWS Lake Formation

In this noteboook, we will show you how to clean up the access control in Lake Formation.The steps will be performed by AWS Lake Formation admin.

##  Clean up

In order to perform the clean up steps, you need to sign in as the **Lake Formation Admin user** you used in notebook *m9_nb0_row_cell_level_access_lf_setup.ipynb*. Detailed instructions can be found here https://docs.aws.amazon.com/lake-formation/latest/dg/getting-started-setup.html#create-data-lake-adminl.


1. Delete feature groups (tables) and features (columns) permissions 
2. delete the required data filters for fined grained access control
3. Deregister the Offline Feature Store in Lake Formation.


### 1) Revoke feature groups (tables) and features (columns) LF permissions

In this section, you will remove the permissions that were granted to the SageMaker user.

In [ ]:
# Retrieve table/FG names (when running previous modules)
%store -r customers_feature_group_name
%store -r orders_feature_group_name
%store -r products_feature_group_name

In [ ]:
# Retrieve the execution role
# Add code to retrieve the S3Uri
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker

sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
role

Repeat for each of the tables in Lake Formation (feature groups) that were granted access in `m9_nb0_row_cell_level_access_lf_setup.ipynb`: 

* In the navigation pane, under **Data Catalog**, choose **Tables**.
* Select table with your `feature group name`.
* Select **View permissions** under the **Actions** button, select `AmazonSageMaker-ExecutionRole-` (associated with your user) and click on **Revoke** button.

![Offline Feature Store Database](../images/fs_lf_revoke_permissions.png "Granular Access using Lake Formation")

### 2) Delete Lake Formation Data Filters

Repeat for the two data filters **Cell Level Filter** and **Row Level Filter** created in `m9_nb0_row_cell_level_access_lf_setup.ipynb`: 
* In the navigation pane, under **Data Catalog**, choose **Data filters**
* Select the data filter by clicking on the radio button
* Click on **Delete** button


![Offline Feature Store Database](../images/fs_lf_create_data_filter_3.png "Granular Access using Lake Formation")

![Offline Feature Store Database](../images/fs_lf_delete_data_filter.png "Granular Access using Lake Formation")

### 3) Deregister Amazon SageMaker Offline Feature Store in Lake Formation

In order to deregister the Amazon S3 location of SageMaker Offline Feature Store, follow the steps:

* In the navigation pane, under **Register and Ingest**, choose **Data lake locations**.
* Select the location of the offline feature store in Amazon S3 (which was used in `m9_nb0_row_cell_level_access_lf_setup.ipynb`)
* Click on **Remove** under **Actions** menu.  

![Offline Feature Store Database](../images/fs_lf_deregister.png "Granular Access using Lake Formation")

To revert back to using IAM instead of the Lake Formation permissions for your Feature Store database and tables, you can add the Super permission back to the **IAMAllowedPrincipals** group on the database in Lake Formation.

* In the navigation pane, under **Data Catalog**, choose **Databases**.
* Select database **sagemaker_featurestore**, which is the database associated to the offline feature store.
* On the **Actions** menu, choose **Edit**.
* On the *Edit database* page, if you want to use only IAM access control for new tables in this database then you can select **Use only IAM access control for new tables in this database**, and then choose **Save**.
* Back on the **Databases** page, ensure that the sagemaker_featurestore database is still selected, and select **View permissions** 
* Click **Grant** and search for **IAMAllowedPrincipals** in the **IAM users and roles** and then grant the appropriate permissions (e.g. Super) on the database and tables.

![Offline Feature Store Database](../images/fs_lf_database.png "Granular Access using Lake Formation")

![Offline Feature Store Database](../images/fs_lf_grant_iam.png "Granular Access using Lake Formation")

### 3) Clean the permissions and policies associated to your SageMaker Execution role

Remove the permissions added in `m9_nb0_row_cell_level_access_lf_setup.ipynb`

{

    "Version": "2012-10-17",
    "Statement": [
    {
        "Sid": "LakeFormationDataAccess",
        "Effect": "Allow",
        "Action": [
            "lakeformation:GetDataAccess",
            "lakeformation:StartQueryPlanning",
            "lakeformation:GetQueryState",
            "lakeformation:GetWorkUnits",
            "lakeformation:GetWorkUnitResults",
            "glue:GetTables",
            "glue:GetDatabases"
        ],
        "Resource": "*"
    }
    ]
}


## Clean up - End